In [83]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
import numpy
import random

In [84]:
def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [85]:
train =[]

In [86]:
for l in readCSV("train_Interactions.csv.gz"):
     train.append(l)

In [87]:
#random.shuffle(train)
dataset_valid = train[190000:]
dataset_train = train[:190000]

In [88]:
dataset_train[0]

['u79354815', 'b14275065', '4']

In [89]:
usersPerBook = defaultdict(list)
booksPerUser = defaultdict(list)
ratingsPerUser = defaultdict(list)
ratingsPerBook = defaultdict(list)

In [90]:
for d in dataset_train :
    user = d[0]
    book = d[1]
    rating = int(d[2])
    usersPerBook[book].append(user)
    booksPerUser[user].append(book)
    ratingsPerUser[user].append(rating)
    ratingsPerBook[book].append(rating)

In [91]:
len(usersPerBook['b13623045'])

65

In [92]:
def srating(ratingsPerUser,u):
    s=0
    for e in ratingsPerUser[u]:
        s += int(e)
        
    return s

In [93]:

ratingMean = sum(int(d[2]) for d in dataset_train) / len(dataset_train)
alpha = ratingMean

In [94]:
userBiases = defaultdict(float)
bookBiases = defaultdict(float)

In [95]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [96]:

def prediction(user, book,userBiases,bookBiases):
    return alpha + userBiases[user] + bookBiases[book]

def cost(userBiases,bookBiases,labels, lamb):
    predictions = [prediction(d[0], d[1],userBiases,bookBiases) for d in dataset_train]
    
    cost = MSE(predictions, labels)
    a=cost
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in bookBiases:
        cost += lamb*bookBiases[i]**2
    return a

##  Fitting the predictor for lambda =1

In [97]:


labels = [int(d[2]) for d in dataset_train]

##initialization    

alpha =  ratingMean

for u in booksPerUser:
    userBiases[u] = ((srating(ratingsPerUser,u))/(len(ratingsPerUser[u])))-ratingMean
    
for i in usersPerBook:
    bookBiases[i] = ((srating(ratingsPerBook,i))/(len(ratingsPerBook[i])))-ratingMean

prev_value = float('inf')

lamb = 4

alphadiff =0
sumR = 0
diffterm =0
N = len(dataset_train)

old_mse = 1
counter = 0
while(True):

    alphadiff = 0
    for d in dataset_train:
        u = d[0]
        i = d[1]
        R = int(d[2])

        alphadiff += (R-userBiases[u]-bookBiases[i])
    alpha = (alphadiff)/N


        ##BETA U    
    for u in userBiases:
        sumR = srating(ratingsPerUser,u)
        diffterm = 0
        for i in booksPerUser[u]:
            diffterm += alpha+bookBiases[i]
        diff = sumR - diffterm
        userBiases[u] = float(diff)/((len(booksPerUser[u]) + lamb))


        ##BETA I
    for i in bookBiases: 
        sumR = srating(ratingsPerBook,i)
        diffterm = 0
        for u in usersPerBook[i]: 
            diffterm += (alpha+userBiases[u])
        diff = sumR - diffterm
        bookBiases[i]= float(diff)/(len(usersPerBook[i]) + lamb)


    mse = cost(userBiases,bookBiases,labels,lamb)
    #print("alpha:", alpha)


    if (old_mse-mse) <= 0.000001:
        break
    else :
        old_mse = mse
    

MSE = 0.9357391107718275
MSE = 0.9286870528352935
MSE = 0.9279491396128474
MSE = 0.9277753325664596
MSE = 0.9277068350048915
MSE = 0.9276694959628146
MSE = 0.9276447436458912
MSE = 0.9276263594961619
MSE = 0.9276118106089392
MSE = 0.9275998918331229
MSE = 0.9275899410526893
MSE = 0.9275815431886733
MSE = 0.9275744087753931
MSE = 0.9275683204026712
MSE = 0.9275631070681037
MSE = 0.9275586305868406
MSE = 0.9275547775261481
MSE = 0.9275514538794956
MSE = 0.9275485812465135
MSE = 0.9275460939350189
MSE = 0.9275439366884799
MSE = 0.9275420628684343
MSE = 0.9275404329844417
MSE = 0.9275390134972087
MSE = 0.9275377758389273
MSE = 0.9275366956090286
MSE = 0.9275357519113228


In [98]:
predict= []
for d in dataset_valid:
    user = d[0]
    book = d[1]
    R = int(d[2])
    r = prediction(user,book,userBiases,bookBiases)
    predict.append(r) 

y_values = [int(d[2]) for d in dataset_valid]
valid_mse = MSE(predict,  y_values)
print("Valid_MSE",valid_mse )

Valid_MSE 1.108786213061298


##  Finding largest and smallest values of beta

In [41]:

beta_B = [(bookBiases[b], b) for b in bookBiases]
beta_B[:3]

[(-0.2405068945977436, 'b85536634'),
 (0.28568050540038126, 'b96213089'),
 (0.5310958145800748, 'b82936372')]

In [42]:
beta_B.sort()
negbeta_B = beta_B[0]
negbeta_B

(-1.7890416452006146, 'b84091840')

In [43]:
beta_B.reverse()
posbeta_B = beta_B[0]
posbeta_B 

(1.1756118796622286, 'b19925500')

Book with greater negative value of beta is b84091840. Book with greater positive value of beta is b19925500

In [44]:
beta_U = [(userBiases[u], u) for u in userBiases]


In [45]:
beta_U.sort()
negbeta_U = beta_U[0]
negbeta_U

(-3.778275374636964, 'u48313610')

In [46]:
beta_U.reverse()
posbeta_U = beta_U[0]
posbeta_U 

(1.3287494174920815, 'u48167710')

User with greater negative value of beta is u80513837. User with greater positive value of beta is u48167710 

##  Finding better value of lambda

In [47]:
def cost(userBiases,bookBiases,labels, lamb):
    predictions = [prediction(d[0], d[1],userBiases,bookBiases) for d in dataset_train]
    
    cost = MSE(predictions, labels)
    a=cost
    #print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in bookBiases:
        cost += lamb*bookBiases[i]**2
    return a

In [65]:
#userBiases = defaultdict(float)
#bookBiases = defaultdict(float)

L=[0.001,0.01,0,1,10]

for lm in L:
    userBiases = defaultdict(float)
    bookBiases = defaultdict(float)
    print("lamba ", lm)
    labels = [int(d[2]) for d in dataset_train]

    ##initialization    

    alpha =  ratingMean

    for u in booksPerUser:
        userBiases[u] = ((srating(ratingsPerUser,u))/(len(ratingsPerUser[u])))-ratingMean

    for i in usersPerBook:
        bookBiases[i] = ((srating(ratingsPerBook,i))/(len(ratingsPerBook[i])))-ratingMean


    lamb = lm

    alphadiff =0
    sumR = 0
    diffterm =0
    N = len(dataset_train)

    old_mse = 1
    counter = 0
    while(True):

        alphadiff = 0
        for d in dataset_train:
            u = d[0]
            i = d[1]
            R = int(d[2])

            alphadiff += (R-userBiases[u]-bookBiases[i])
        alpha = (alphadiff)/N


            ##BETA U    
        for u in userBiases:
            sumR = srating(ratingsPerUser,u)
            diffterm = 0
            for i in booksPerUser[u]:
                diffterm += alpha+bookBiases[i]
            diff = sumR - diffterm
            userBiases[u] = float(diff)/((len(booksPerUser[u]) + lamb))


            ##BETA I
        for i in bookBiases: 
            sumR = srating(ratingsPerBook,i)
            diffterm = 0
            for u in usersPerBook[i]: 
                diffterm += (alpha+userBiases[u])
            diff = sumR - diffterm
            bookBiases[i]= float(diff)/(len(usersPerBook[i]) + lamb)


        mse = cost(userBiases,bookBiases,labels,lamb)
        #print("alpha:", alpha)
         

        if (old_mse-mse) <= 0.000001:
            
            predict= []
            for d in dataset_valid:
                user = d[0]
                book = d[1]
                R = int(d[2])
                r = prediction(user,book,userBiases,bookBiases)
                predict.append(r) 

            y_values = [int(d[2]) for d in dataset_valid]
            valid_mse = MSE(predict,  y_values)
            print("Valid_MSE",valid_mse )
            
            break
        else :
            old_mse = mse


lamba  0.001
MSE = 0.9080603499507711
MSE = 0.9040204717610901
MSE = 0.9035548684740015
MSE = 0.9034290244712353
MSE = 0.9033866626100694
MSE = 0.9033715850600921
MSE = 0.9033661278594212
MSE = 0.9033641397647996
MSE = 0.9033634125206206
Valid_MSE 1.0818490304127444
lamba  0.01
MSE = 0.9080840495103893
MSE = 0.904026759499787
MSE = 0.9035577366560468
MSE = 0.9034308114878115
MSE = 0.9033879155438456
MSE = 0.9033725100157434
MSE = 0.903366840465744
MSE = 0.9033647153835761
MSE = 0.9033639011320657
Valid_MSE 1.0816620849062084
lamba  0
MSE = 0.9080577410638572
MSE = 0.9040198037873162
MSE = 0.9035545824544984
MSE = 0.903428859433095
MSE = 0.9033865574137799
MSE = 0.9033715166484891
MSE = 0.903366083286035
MSE = 0.9033641105841043
MSE = 0.9033633931261859
Valid_MSE 1.0818698801533198
lamba  1
MSE = 0.9126449188106961
MSE = 0.9071587490840035
MSE = 0.9064597212196943
MSE = 0.9062719030301315
MSE = 0.9062021168192123
MSE = 0.9061711415345614
MSE = 0.9061557409017105
MSE = 0.9061474208230353

## Predictinf the ratings
lambda = 1 yields better error rate for the validation set. MSE of validation set for lambda=1 is 1.1191


In [99]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    #x=str(u)
    #print(x)
    #if user in userBiases and book in bookBiases :
    r = prediction(u,b,userBiases,bookBiases)
    #else:
        #r=0
    predictions.write(u + '-' + b + ',' + str(r) + '\n')
predictions.close()